In [ ]:
!pip install ipywidgets

**Simulation code but with atoms as spheres**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interact, IntSlider

# Constants
sigma = 0.34  # nm (Lennard-Jones distance parameter)
epsilon = 120 * 1.38e-23  # J (Lennard-Jones energy parameter)
mass = 39.948 * 1.66e-27  # kg (Argon atomic mass)
kb = 1.38e-23  # J/K (Boltzmann constant)
dt = 1e-15  # s (Time step)
num_steps = 1000  # Reduced for practical execution
cutoff = 2.5 * sigma  # Lennard-Jones cutoff distance

# Box size (Fixed at 3nm × 3nm × 3nm)
box_size = 3.0  # nm

# Initialize Positions (Efficiently ensuring interaction range)
np.random.seed(42)
position1 = np.random.uniform(0.5, box_size - 0.5, (3,))

# Generate a random unit vector
random_direction = np.random.normal(0, 1, (3,))
random_direction /= np.linalg.norm(random_direction)  # Normalize to unit length

# Scale by a random distance between 1.1σ and 1.8σ
distance = np.random.uniform(1.05 * sigma, 1.1 * sigma)
position2 = position1 + random_direction * distance

# Apply periodic boundary conditions (PBCs)
position2 = position2 % box_size

# Initial velocities from Maxwell-Boltzmann distribution (Realistic)
temperature = 300  # K
std_dev = np.sqrt(kb * temperature / mass)
velocity1 = np.random.normal(0, std_dev, (3,))
velocity2 = np.random.normal(0, std_dev, (3,))

# Store trajectories
positions1_list = [position1.copy()]
positions2_list = [position2.copy()]
velocities1_list = [velocity1.copy()]
velocities2_list = [velocity2.copy()]

# Define Functions
def minimum_image_distance(r1, r2, box_size):
    delta_r = r1 - r2
    delta_r -= box_size * np.round(delta_r / box_size)
    return delta_r, np.linalg.norm(delta_r)

def lennard_jones_force(r1, r2, box_size):
    delta_r, r = minimum_image_distance(r1, r2, box_size)
    if r > cutoff:
        return np.zeros(3), np.zeros(3)
    force_mag = 24 * epsilon * (2 * (sigma / r) ** 12 - (sigma / r) ** 6) / r ** 2
    force = force_mag * delta_r / r
    return force, -force

# Debug Lennard-Jones forces at t=0
force1, force2 = lennard_jones_force(position1, position2, box_size)
print(f"Initial Force on Atom 1: {force1}")
print(f"Initial Force on Atom 2: {force2}")

# Run MD Simulation
for step in range(num_steps):
    force1, force2 = lennard_jones_force(position1, position2, box_size)

    new_position1 = position1 + velocity1 * dt + 0.5 * (force1 / mass) * dt ** 2
    new_position2 = position2 + velocity2 * dt + 0.5 * (force2 / mass) * dt ** 2

    new_position1 = new_position1 % box_size
    new_position2 = new_position2 % box_size

    new_force1, new_force2 = lennard_jones_force(new_position1, new_position2, box_size)

    velocity1 += 0.5 * (force1 + new_force1) / mass * dt
    velocity2 += 0.5 * (force2 + new_force2) / mass * dt

    positions1_list.append(new_position1.copy())
    positions2_list.append(new_position2.copy())
    velocities1_list.append(velocity1.copy())
    velocities2_list.append(velocity2.copy())

    position1 = new_position1
    position2 = new_position2

positions1_array = np.array(positions1_list)
positions2_array = np.array(positions2_list)
velocities1_array = np.array(velocities1_list)
velocities2_array = np.array(velocities2_list)

# Function to Plot Atomic Volume Instead of Just Points and Print Velocities & Positions
def plot_trajectory_with_volume(max_timestep):
    step_size = 10
    selected_positions1 = positions1_array[:max_timestep:step_size]
    selected_positions2 = positions2_array[:max_timestep:step_size]

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    def plot_sphere(ax, center, radius, color):
        u = np.linspace(0, 2 * np.pi, 15)
        v = np.linspace(0, np.pi, 15)
        x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
        y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
        z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color=color, alpha=0.3)

    # Plot Atom 1 with Volume
    for pos in selected_positions1:
        plot_sphere(ax, pos, sigma/2, 'blue')

    # Plot Atom 2 with Volume
    for pos in selected_positions2:
        plot_sphere(ax, pos, sigma/2, 'red')

    ax.set_xlim(0, box_size)
    ax.set_ylim(0, box_size)
    ax.set_zlim(0, box_size)
    ax.set_xlabel("X Position (nm)")
    ax.set_ylabel("Y Position (nm)")
    ax.set_zlabel("Z Position (nm)")
    ax.set_title(f"Trajectory of Two Argon Atoms with Atomic Volume (0 to {max_timestep})")
    plt.show()

    # Print positions and velocities for the selected timestep
    timestep_index = max_timestep - 1
    print(f"\nTimestep: {max_timestep}")
    print(f"Position of Atom 1: {positions1_array[timestep_index]}")
    print(f"Velocity of Atom 1: {velocities1_array[timestep_index]}")
    print(f"Position of Atom 2: {positions2_array[timestep_index]}")
    print(f"Velocity of Atom 2: {velocities2_array[timestep_index]}\n")

# Interactive Visualization
interact(plot_trajectory_with_volume, max_timestep=IntSlider(min=10, max=num_steps, step=10, value=100));


Initial Force on Atom 1: [ 3.72173914e-20 -1.06744488e-20 -9.34020566e-21]
Initial Force on Atom 2: [-3.72173914e-20  1.06744488e-20  9.34020566e-21]


interactive(children=(IntSlider(value=100, description='max_timestep', max=1000, min=10, step=10), Output()), …

Timestep: 10
Position of Atom 1: [1.24908024 2.40142861 1.96398788]
Velocity of Atom 1: [ 252.48843646 -117.30337525  135.56463629]
Position of Atom 2: [0.90423752 2.50033415 2.05053081]
Velocity of Atom 2: [-115.79004343 -116.36773735   60.45695354]

**Current timestep and entire trajectory visualization**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interact, IntSlider

# Constants
sigma = 0.34  # nm
epsilon = 120 * 1.38e-23  # J
mass = 39.948 * 1.66e-27  # kg
kb = 1.38e-23  # J/K
dt = 1e-15  # s
num_steps = 1000  # Reduced for quick debugging
cutoff = 2.5 * sigma

# Box size
box_size = 1.0  # nm

# Initialize Positions ensuring interaction
np.random.seed(42)
position1 = np.random.uniform(0.5, box_size - 0.5, (3,))

# Generate random unit vector for Atom 2 placement
random_direction = np.random.normal(0, 1, (3,))
random_direction /= np.linalg.norm(random_direction)
distance = np.random.uniform(1.1 * sigma, 1.5 * sigma)  # Reduce to 1.5σ max for stronger force
position2 = position1 + random_direction * distance
position2 = position2 % box_size  # Apply PBCs

# Initial velocities
temperature = 300  # K
std_dev = np.sqrt(kb * temperature / mass)
velocity1 = np.random.normal(0, std_dev, (3,))
velocity2 = np.random.normal(0, std_dev, (3,))

# Store trajectories
positions1_list = [position1.copy()]
positions2_list = [position2.copy()]
velocities1_list = [velocity1.copy()]
velocities2_list = [velocity2.copy()]

# Define Functions
def minimum_image_distance(r1, r2, box_size):
    delta_r = r1 - r2
    delta_r -= box_size * np.round(delta_r / box_size)
    return delta_r, np.linalg.norm(delta_r)

def lennard_jones_force(r1, r2, box_size):
    delta_r, r = minimum_image_distance(r1, r2, box_size)
    if r > cutoff:
        return np.zeros(3), np.zeros(3)
    force_mag = 24 * epsilon * (2 * (sigma / r) ** 12 - (sigma / r) ** 6) / r ** 2
    force = force_mag * delta_r / r
    return force, -force

# Print Initial Conditions
force1, force2 = lennard_jones_force(position1, position2, box_size)
print(f"Initial Distance: {np.linalg.norm(position1 - position2):.4f} nm")
print(f"Initial Force on Atom 1: {force1}")
print(f"Initial Force on Atom 2: {force2}")
print(f"Initial Velocity of Atom 1: {velocity1}")
print(f"Initial Velocity of Atom 2: {velocity2}")

# Run MD Simulation
for step in range(num_steps):
    force1, force2 = lennard_jones_force(position1, position2, box_size)

    new_position1 = position1 + velocity1 * dt + 0.5 * (force1 / mass) * dt ** 2
    new_position2 = position2 + velocity2 * dt + 0.5 * (force2 / mass) * dt ** 2

    new_position1 = new_position1 % box_size
    new_position2 = new_position2 % box_size

    new_force1, new_force2 = lennard_jones_force(new_position1, new_position2, box_size)

    velocity1 += 0.5 * (force1 + new_force1) / mass * dt
    velocity2 += 0.5 * (force2 + new_force2) / mass * dt

    positions1_list.append(new_position1.copy())
    positions2_list.append(new_position2.copy())
    velocities1_list.append(velocity1.copy())
    velocities2_list.append(velocity2.copy())

    position1 = new_position1
    position2 = new_position2

    # Debugging: Print forces at regular intervals
    if step % 1000 == 0:
        print(f"Step {step}: Distance = {np.linalg.norm(position1 - position2):.4f} nm")
        print(f"Force on Atom 1: {force1}")
        print(f"Velocity of Atom 1: {velocity1}\n")

positions1_array = np.array(positions1_list)
positions2_array = np.array(positions2_list)
velocities1_array = np.array(velocities1_list)
velocities2_array = np.array(velocities2_list)

# Function to plot full trajectory and current frame
def plot_trajectory_with_current_frame(max_timestep):
    step_size = 10
    selected_positions1 = positions1_array[:max_timestep:step_size]
    selected_positions2 = positions2_array[:max_timestep:step_size]

    fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

    def plot_sphere(ax, center, radius, color):
        u = np.linspace(0, 2 * np.pi, 15)
        v = np.linspace(0, np.pi, 15)
        x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
        y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
        z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color=color, alpha=0.3)

    # Left plot: Full trajectory
    ax1 = axes[0]
    ax1.plot(selected_positions1[:, 0], selected_positions1[:, 1], selected_positions1[:, 2],
             marker='o', linestyle='-', color='blue', label="Atom 1 Trajectory")
    ax1.plot(selected_positions2[:, 0], selected_positions2[:, 1], selected_positions2[:, 2],
             marker='o', linestyle='-', color='red', label="Atom 2 Trajectory")
    ax1.set_title(f"Full Trajectory (0 to {max_timestep})")
    ax1.set_xlabel("X (nm)")
    ax1.set_ylabel("Y (nm)")
    ax1.set_zlabel("Z (nm)")
    ax1.legend()

    # Right plot: Only current frame
    ax2 = axes[1]
    plot_sphere(ax2, positions1_array[max_timestep], sigma/2, 'blue')
    plot_sphere(ax2, positions2_array[max_timestep], sigma/2, 'red')
    ax2.set_xlim(0, box_size)
    ax2.set_ylim(0, box_size)
    ax2.set_zlim(0, box_size)
    ax2.set_title(f"Current Frame at Timestep {max_timestep}")
    ax2.set_xlabel("X (nm)")
    ax2.set_ylabel("Y (nm)")
    ax2.set_zlabel("Z (nm)")

    plt.show()

    # Print positions and velocities for the selected timestep
    print(f"\nTimestep: {max_timestep}")
    print(f"Position of Atom 1: {positions1_array[max_timestep]}")
    print(f"Velocity of Atom 1: {velocities1_array[max_timestep]}")
    print(f"Position of Atom 2: {positions2_array[max_timestep]}")
    print(f"Velocity of Atom 2: {velocities2_array[max_timestep]}\n")

# Interactive Visualization
interact(plot_trajectory_with_current_frame, max_timestep=IntSlider(min=10, max=num_steps, step=10, value=100));

Initial Distance: 0.4703 nm
Initial Force on Atom 1: [-1.71273581e-20  4.91235685e-21  4.29834121e-21]
Initial Force on Atom 2: [ 1.71273581e-20 -4.91235685e-21 -4.29834121e-21]
Initial Velocity of Atom 1: [ 252.48843641 -117.30337524  135.5646363 ]
Initial Velocity of Atom 2: [-115.79004338 -116.36773737   60.45695353]
Step 0: Distance = 0.4860 nm
Force on Atom 1: [-1.71273581e-20  4.91235685e-21  4.29834121e-21]
Velocity of Atom 1: [ 240.67799792 -113.97727135  138.43054778]



interactive(children=(IntSlider(value=100, description='max_timestep', max=1000, min=10, step=10), Output()), …

**Printing out the Force, Velocity and Position of both atoms**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interact, IntSlider

# Constants
sigma = 0.34  # nm
epsilon = 120 * 1.38e-23  # J
mass = 39.948 * 1.66e-27  # kg
kb = 1.38e-23  # J/K
dt = 5e-15  # s
num_steps = 100000  # Reduced for quick debugging
cutoff = 1.5

# Box size
box_size = 3.0  # nm

# Initialize Positions ensuring interaction
np.random.seed(42)
position1 = np.random.uniform(0.5, box_size - 0.5, (3,))

# Generate random unit vector for Atom 2 placement
random_direction = np.random.normal(0, 1, (3,))
random_direction /= np.linalg.norm(random_direction)
distance = np.random.uniform(1.1 * sigma, 1.5 * sigma)  # Reduce to 1.5σ max for stronger force
position2 = position1 + random_direction * distance
position2 = position2 % box_size  # Apply PBCs

# Initial velocities
temperature = 300  # K
std_dev = np.sqrt(kb * temperature / mass)
velocity1 = np.random.normal(0, std_dev, (3,)) * 10  # Increased velocity
velocity2 = np.random.normal(0, std_dev, (3,)) * 10  # Increased velocity

# Store trajectories
positions1_list = [position1.copy()]
positions2_list = [position2.copy()]
velocities1_list = [velocity1.copy()]
velocities2_list = [velocity2.copy()]

# Define Functions
def minimum_image_distance(r1, r2, box_size):
    delta_r = r1 - r2
    delta_r -= box_size * np.round(delta_r / box_size)
    return delta_r, np.linalg.norm(delta_r)

def lennard_jones_force(r1, r2, box_size):
    delta_r, r = minimum_image_distance(r1, r2, box_size)
    if r > cutoff:
        return np.zeros(3), np.zeros(3)
    force_mag = 24 * epsilon * (2 * (sigma / r) ** 12 - (sigma / r) ** 6) / r ** 2
    force = force_mag * delta_r / r
    return force, -force

# Print Initial Conditions
force1, force2 = lennard_jones_force(position1, position2, box_size)
print(f"Initial Distance: {np.linalg.norm(position1 - position2):.4f} nm")
print(f"Initial Force on Atom 1: {force1}")
print(f"Initial Force on Atom 2: {force2}")
print(f"Initial Velocity of Atom 1: {velocity1}")
print(f"Initial Velocity of Atom 2: {velocity2}")

# Run MD Simulation
for step in range(num_steps):
    force1, force2 = lennard_jones_force(position1, position2, box_size)

    new_position1 = position1 + velocity1 * dt
    new_position2 = position2 + velocity2 * dt

    new_position1 = new_position1 % box_size
    new_position2 = new_position2 % box_size

    new_force1, new_force2 = lennard_jones_force(new_position1, new_position2, box_size)

    velocity1 += 0.5 * (force1 + new_force1) / mass * dt
    velocity2 += 0.5 * (force2 + new_force2) / mass * dt

    positions1_list.append(new_position1.copy())
    positions2_list.append(new_position2.copy())
    velocities1_list.append(velocity1.copy())
    velocities2_list.append(velocity2.copy())

    position1 = new_position1
    position2 = new_position2

    # Debugging: Print forces at regular intervals
    if step % 1000 == 0:
        print(f"Step {step}: Distance = {np.linalg.norm(position1 - position2):.4f} nm")
        print(f"Force on Atom 1: {force1}")
        print(f"Velocity of Atom 1: {velocity1}\n")

positions1_array = np.array(positions1_list)
positions2_array = np.array(positions2_list)

# Function to plot full trajectory and current frame
def plot_trajectory_with_current_frame(max_timestep):
    step_size = 10
    selected_positions1 = positions1_array[:max_timestep:step_size]
    selected_positions2 = positions2_array[:max_timestep:step_size]

    fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

    def plot_sphere(ax, center, radius, color):
        u = np.linspace(0, 2 * np.pi, 15)
        v = np.linspace(0, np.pi, 15)
        x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
        y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
        z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color=color, alpha=0.3)

    # Left plot: Full trajectory
    ax1 = axes[0]
    ax1.plot(selected_positions1[:, 0], selected_positions1[:, 1], selected_positions1[:, 2],
             marker='o', linestyle='-', color='blue', label="Atom 1 Trajectory")
    ax1.plot(selected_positions2[:, 0], selected_positions2[:, 1], selected_positions2[:, 2],
             marker='o', linestyle='-', color='red', label="Atom 2 Trajectory")
    ax1.set_title(f"Full Trajectory (0 to {max_timestep})")
    ax1.set_xlabel("X (nm)")
    ax1.set_ylabel("Y (nm)")
    ax1.set_zlabel("Z (nm)")
    ax1.legend()

    # Right plot: Only current frame
    ax2 = axes[1]
    plot_sphere(ax2, positions1_array[max_timestep], sigma/2, 'blue')
    plot_sphere(ax2, positions2_array[max_timestep], sigma/2, 'red')
    ax2.set_xlim(0, box_size)
    ax2.set_ylim(0, box_size)
    ax2.set_zlim(0, box_size)
    ax2.set_title(f"Current Frame at Timestep {max_timestep}")
    ax2.set_xlabel("X (nm)")
    ax2.set_ylabel("Y (nm)")
    ax2.set_zlabel("Z (nm)")

    plt.show()

    # Print positions and velocities for the selected timestep
    print(f"\nTimestep: {max_timestep}")
    print(f"Position of Atom 1: {positions1_array[max_timestep]}")
    print(f"Velocity of Atom 1: {velocities1_array[max_timestep]}")
    print(f"Position of Atom 2: {positions2_array[max_timestep]}")
    print(f"Velocity of Atom 2: {velocities2_array[max_timestep]}\n")

# Interactive Visualization
interact(plot_trajectory_with_current_frame, max_timestep=IntSlider(min=10, max=num_steps, step=10, value=100));


Initial Distance: 0.4703 nm
Initial Force on Atom 1: [-1.71273581e-20  4.91235685e-21  4.29834121e-21]
Initial Force on Atom 2: [ 1.71273581e-20 -4.91235685e-21 -4.29834121e-21]
Initial Velocity of Atom 1: [ 2524.88436407 -1173.03375238  1355.64636301]
Initial Velocity of Atom 2: [-1157.90043377 -1163.67737366   604.56953532]
Step 0: Distance = 0.4703 nm
Force on Atom 1: [-1.71273581e-20  4.91235685e-21  4.29834121e-21]
Velocity of Atom 1: [ 2524.88436406 -1173.03375238  1355.64636301]

Step 1000: Distance = 0.4703 nm
Force on Atom 1: [-1.71273549e-20  4.91235573e-21  4.29834008e-21]
Velocity of Atom 1: [ 2524.88436277 -1173.03375201  1355.64636334]

Step 2000: Distance = 0.4703 nm
Force on Atom 1: [-1.71273517e-20  4.91235460e-21  4.29833895e-21]
Velocity of Atom 1: [ 2524.88436148 -1173.03375164  1355.64636366]

Step 3000: Distance = 0.4703 nm
Force on Atom 1: [-1.71273485e-20  4.91235348e-21  4.29833782e-21]
Velocity of Atom 1: [ 2524.88436019 -1173.03375127  1355.64636399]

Step 40

interactive(children=(IntSlider(value=100, description='max_timestep', max=100000, min=10, step=10), Output())…